In [9]:
import numpy as np
%matplotlib qt 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import pickle
import utils

In [10]:
image1_path = 'frame1.png'
image2_path = 'frame_1.png'


image1_points = np.loadtxt(f'{image1_path[:-4]}_undistorted_image_points.txt')
image1_points = np.array(image1_points, dtype=np.float32)
image2_points = np.loadtxt(f'{image2_path[:-4]}_undistorted_image_points.txt')
image2_points = np.array(image2_points, dtype=np.float32)
world_points = np.loadtxt('world_points_3D.txt')
world_points = np.array(world_points, dtype=np.float32)
image1_camera = pickle.load(open(f'{image1_path[:-4]}_camera_object.pkl', 'rb'))
image2_camera = pickle.load(open(f'{image2_path[:-4]}_camera_object.pkl', 'rb'))


In [11]:
# Triangulate points 

triangulated_points = utils.triangulate(image1_points, image1_camera, image2_points, image2_camera)

# Plot 3D points

utils.visualize_triangulated_points(triangulated_points, world_points)

# Get triangulation error

print('Triangulation error: ', np.linalg.norm(triangulated_points - world_points, axis=0).mean())
    

Triangulation error:  0.6478642706230918


In [4]:
# Select new points to triangulate

image1 = cv2.imread(image1_path)
# Interactive window to collect points from the image
image1_selected = []
image1_selected = utils.interactive_clicking_and_saving(image1, image1_selected)


In [5]:

utils.display_image_with_points(image1, image1_selected)

In [6]:
# Select new points to triangulate

image2 = cv2.imread(image2_path)
# Interactive window to collect points from the image
image2_selected = []
image2_selected = utils.interactive_clicking_and_saving(image2, image2_selected)

In [7]:
utils.display_image_with_points(image2, image2_selected)

In [8]:
# Dewarp both points

dewarped_topdown_selected = cv2.undistortPoints(np.array([image1_selected]), image1_camera.camera_matrix, image1_camera.distortion_coefficients, P=image1_camera.new_camera_matrix).reshape(-1,2)
dewarped_fronton_selected = cv2.undistortPoints(np.array([image2_selected]), image2_camera.camera_matrix, image2_camera.distortion_coefficients, P=image2_camera.new_camera_matrix).reshape(-1,2)

triangulated_points = utils.triangulate(dewarped_topdown_selected, image1_camera, dewarped_fronton_selected, image2_camera)

# Visualize the new triangulated points
NET_Y = 0.76 + 3.96 + 1.98
NET_HEIGHT = 1.55
RIGHT_BOUNDARY_X = 0.46 + 2.59 + 2.59 + 0.46
new_world_points = np.array([[0 ,NET_Y, NET_HEIGHT],[ RIGHT_BOUNDARY_X,NET_Y ,NET_HEIGHT]])

utils.visualize_triangulated_points(triangulated_points, new_world_points)

# Get triangulation error

print('Triangulation error: ', np.linalg.norm(triangulated_points - new_world_points, axis=0).mean())


Triangulation error:  0.3646356566856905
